In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import  naive_bayes

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

In [2]:
# Defining feature extraction and preprocessing steps
text_feature = 'text'
categorical_features = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

# Splitting data into train and test sets
train=pd.read_csv('../Datasets/fake-news/train.csv')
test=pd.read_csv('../Datasets/fake-news/test.csv')

In [3]:
train=train[~train['text'].isna()]

In [4]:
snowballStemmer = SnowballStemmer("english")
stop= stopwords.words("english")

def tokenize(text):
    return text.split()
def tokenize_snowball(text):
    return [snowballStemmer.stem(word) for word in text.split()]

In [ ]:
pipeline = Pipeline([("vect", TfidfVectorizer()),("clf", naive_bayes.MultinomialNB())])
params = [{"vect__ngram_range": [(1,1), (1,2)],
              "vect__stop_words": [None, stop],
              "vect__tokenizer": [tokenize,
                                  tokenize_snowball],
              "clf__alpha": [0.01, 0.1, 0.5, 1.0, 3.0, 10.0]}]

grid = GridSearchCV(estimator=pipeline, param_grid = params ,scoring='accuracy', cv=5, verbose=1, n_jobs=4)
grid.fit(train['text'],train['label'])
model1 = grid.best_estimator_

In [15]:
test=pd.merge(test,pd.read_csv('../Datasets/fake-news/submit.csv'), left_on='id', right_on='id')
test=test[~test['text'].isna()]
test['pred_NB1']= model1.predict(test['text'])
print(accuracy_score(test['label'], test['pred_NB1']))

0.6131330637396495


In [17]:
test.to_csv('../Datasets/fake-news/pred_NB.csv')

In [18]:
best_params = grid.best_params_

# Print the best parameters
print("Best Parameters:", best_params)

Best Parameters: {'clf__alpha': 0.01, 'vect__ngram_range': (1, 1), 'vect__stop_words': ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'mos

In [52]:
from joblib import dump
dump(model1, '../models/NB.joblib')


['../models/NB.joblib']

# Adding other features

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [20]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [57]:
# Defining feature extraction and preprocessing steps
text_feature = 'text'
categorical_features = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise','word_count', 'stopword_count', 'prop_stop']


train=pd.read_csv('../Datasets/fake-news/train.csv')
train=train[(~train['text'].isna())|(~train['title'].isna())]
# Splitting data into train and test sets
train=pd.merge(train,pd.read_csv('../Datasets/fake-news/scores_emotion.csv'), right_on='id', left_on='id')
train=pd.merge(train,pd.read_csv('../Datasets/fake-news/stop_words.csv'), right_on='id', left_on='id')
test=pd.read_csv('../Datasets/fake-news/test.csv')
test=test[~test['text'].isna()|~test['title'].isna()]
test=pd.merge(test,pd.read_csv('../Datasets/fake-news/scores_emotion_test.csv'), right_on='id', left_on='id')
test=pd.merge(test,pd.read_csv('../Datasets/fake-news/stop_words_test.csv'), right_on='id', left_on='id')
train = train.drop('author', axis=1)
test = test.drop('author', axis=1)
train=train[train['word_count']>0]
test=test[test['word_count']>0]
test=pd.merge(test,pd.read_csv('../Datasets/fake-news/submit.csv'), left_on='id', right_on='id')


In [58]:
# Build a ColumnTransformer with FeatureUnion
text_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 1), stop_words=stop))
])

num_pipeline = Pipeline([
    ('discretizer', KBinsDiscretizer(n_bins=3, encode='onehot-dense', strategy='kmeans'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, categorical_features),
    ('text', text_pipeline, text_feature)
])

# Define the Naive Bayes classifier
naive_bayes = MultinomialNB(alpha=0.01)

# Creating the final pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('naive_bayes', naive_bayes)
])
print('fitting')
# Fitting the pipeline on the training data
pipeline.fit(train,train['label'])
print('fitted')

# Predicting on the test data
test['pred'] = pipeline.predict(test)

# Evaluating the performance
print(accuracy_score(test['label'], test['pred']))


fitting
fitted
0.6275793650793651


In [24]:
preprocessor.transform(X_test)

<8980x3386908 sparse matrix of type '<class 'numpy.float64'>'
	with 4926382 stored elements in Compressed Sparse Row format>

In [59]:
dump(pipeline, '../models/NB_2.joblib')


['../models/NB_2.joblib']

In [60]:
test.to_csv('../Datasets/fake-news/pred_NB2.csv')